In [1]:
#Importing all necessary libraries

import pandas, numpy, os
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import findspark
findspark.init()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
!pyspark --version

In [ ]:
#Creating a Spark Session

conf = SparkConf()\
    .setAppName("Purush_ETL")\
    .setMaster("local[2]")

spark = SparkSession\
    .builder\
    .config(conf = conf)\
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.instances", "2")\
    .config("spark.executor.cores", "5")\
    .config("spark.driver.memory", "2g")\
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.dynamicAllocation.minExecutors","1")\
    .config("spark.dynamicAllocation.maxExecutors","3")\
    .enableHiveSupport()\
    .getOrCreate()

In [ ]:
pathforDWH = "/user/hive/warehouse/test"
pathforDL = "s3a://dwanddl/user/"

In [ ]:
# For Data Warehouse
dwh = spark.read.parquet(pathforDWH)

dwh.printSchema() #to check the Schema of the dataframe
#dwh.show()

In [ ]:
# For Data Lake
dl = spark.read.parquet(pathforDL)

dl.printSchema() #to check the Schema of the dataframe
#dl.show()

In [ ]:
#Join

consolidated_df = dwh.union(dl) #picks onty distinct records

In [ ]:
consolidated_df.repartion('Month').write.mode("overwrite").format("parquet").path(pathforDL)

In [ ]:
##Query
df = spark.read.parquet(pathforDL)
df.createOrReplaceTempView("dbtable")
df.spark.sql("SELECT Month, COUNT(), COUNT() FROM dbtable WHERE isStudent = "False" GROUP BY Month")

In [ ]:
##Query

SELECT 
    Month,
    COUNT(CASE WHEN minsalary = 1 THEN 1 ELSE NULL END) AS lowsalarycount,
    COUNT(CASE WHEN maxsalary = 1 THEN 1 ELSE NULL END) AS highsalarycount
FROM (
SELECT
    Month,
    ROW_NUMBER() OVER (PARTITION BY Month ORDER BY Salary DESC) AS maxsalary,
    ROW_NUMBER() OVER (PARTITION BY Month ORDER BY Salary ASC) AS minsalary,
    isStudent = 'False'
)
GROUP BY Month

In [ ]:
spark.stop()